In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np
import time
from datetime import datetime

In [2]:
q = 0
last_trial_time = datetime.now()
while q < 1:
    
    print(last_trial_time)
    control_dataframe = pd.read_csv('control.csv' ) 
    
    if control_dataframe['corelation'].iloc[:1].values == 1:     
        
        q = q+1
        
        
        
    else:  
        last_trial_time = datetime.now()
        time.sleep(60)
        
    

2021-12-13 22:17:02.743869


In [3]:
print('---------------- Script Has Started Execution-----------------------')

currency_pairs = [
                    'USDCHF10080',
                 'GBPUSD10080',
                  'EURUSD10080', 
    'USDJPY10080',
    'USDCAD10080', 'AUDUSD10080', 'NZDUSD10080',
                 'GBPCHF10080', 'EURCHF10080', 'CHFJPY10080', 'CADCHF10080', 'AUDCHF10080', 'NZDCHF10080', 'EURGBP10080',
                 'GBPJPY10080', 'GBPCAD10080', 'GBPAUD10080', 'EURJPY10080', 'EURCAD10080', 'EURAUD10080', 'EURNZD10080',
                'CADJPY10080', 'AUDJPY10080', 'NZDJPY10080', 'AUDCAD10080', 'NZDCAD10080', 'AUDNZD10080'
                ]

headers = ['date', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]


# load the corelation data from CSV
correlation_dataframe = pd.read_csv('files/auto-correlation2.csv'  ) 

# declair the variable that will hold all testable units, the week and the needed feature
currency_pair_detail = []

---------------- Script Has Started Execution-----------------------


In [4]:

# loop through all currency pairs
for a in currency_pairs:
    
    current_currency = a.replace('10080','')
    
    
    weeks = [    {'week_name' : 10, 'correlation_cutoff' : 0.632},
                 {'week_name' : 15, 'correlation_cutoff' : 0.514},
                 {'week_name' : 20, 'correlation_cutoff' : 0.444},
                 {'week_name' : 30, 'correlation_cutoff' : 0.361},
                 {'week_name' : 50, 'correlation_cutoff' : 0.279},
            ]
    
   
    #loop throuh all available weeks
    for b in weeks:
        
        needed_column_list = []
        
        #check if the current currency in the current week has any currency above the correlation score of the week
        #this is to avoid running analysis for weeks with no correlation
        if  correlation_dataframe[current_currency + '_' + str(b['week_name'])][0] > b['correlation_cutoff']:
            
            #loop though weeks to get all the currency pairs in the weeks lesser than current week according to my strategy
            for c in weeks:

                if c['week_name'] <= b['week_name']:

                    #gets the needed two needed columns
                    current_needed_columns = correlation_dataframe[[current_currency + '__' + str(c['week_name']), current_currency + '_' + str(c['week_name'])]] 
                    
                    #get the rows that has corelation
                    current_needed_columns = current_needed_columns.loc[ (current_needed_columns[current_currency + '_' + str(c['week_name'])] >= c['correlation_cutoff'])]
                    
                    #converts the currency pairs that passes to list
                    current_needed_columns = current_needed_columns[current_currency + '__' + str(c['week_name'])].values.tolist()
                    
                    #loop through all the currency pairs that passes and append each currency pair to the list if it is not there already
                    for d in current_needed_columns :
                        if d not in needed_column_list:
                            needed_column_list.append(d) 


        #creates a dictionary with the needed info for the currency and week
        current_currency_detail = {'currency_name': current_currency, 'week': b['week_name'], 'needed_features':needed_column_list }
        
        #appends all non empty currency detail to the global list
        if len(current_currency_detail['needed_features']) > 0:
            currency_pair_detail.append(current_currency_detail)
        
       
    
        




print(currency_pair_detail)


[{'currency_name': 'GBPUSD', 'week': 10, 'needed_features': ['USDJPY']}, {'currency_name': 'GBPUSD', 'week': 15, 'needed_features': ['USDJPY', 'CHFJPY']}, {'currency_name': 'GBPUSD', 'week': 20, 'needed_features': ['USDJPY', 'CHFJPY', 'GBPCAD', 'AUDJPY']}, {'currency_name': 'GBPUSD', 'week': 30, 'needed_features': ['USDJPY', 'CHFJPY', 'GBPCAD', 'AUDJPY']}, {'currency_name': 'GBPUSD', 'week': 50, 'needed_features': ['USDJPY', 'CHFJPY', 'GBPCAD', 'AUDJPY', 'EURGBP', 'GBPAUD', 'NZDCHF', 'EURCHF']}, {'currency_name': 'USDCAD', 'week': 15, 'needed_features': ['EURGBP']}, {'currency_name': 'USDCAD', 'week': 20, 'needed_features': ['EURGBP', 'EURCAD', 'CHFJPY', 'EURAUD']}, {'currency_name': 'USDCAD', 'week': 30, 'needed_features': ['EURGBP', 'EURCAD', 'CHFJPY', 'EURAUD']}, {'currency_name': 'USDCAD', 'week': 50, 'needed_features': ['EURGBP', 'EURCAD', 'CHFJPY', 'EURAUD', 'EURCHF']}, {'currency_name': 'AUDUSD', 'week': 50, 'needed_features': ['EURCHF']}, {'currency_name': 'NZDUSD', 'week': 20,

In [5]:
for e in currency_pair_detail:
    
    print (e['currency_name'])
    current_currency = e['currency_name']

    raw_data = pd.read_csv('files/' + e['currency_name'] + '.csv'  ) 
    weeks = e['week']

    number_of_accurate_prediction = 0
    number_of_trades = 0


    step_down = 5
    cut_off = 0
    n_estimators = 20000
    min_samples_leaf = 1
    max_features = "auto"
    max_samples = None



    for v in range(step_down):

      


        data = raw_data.copy()
        data = data[v:v+weeks]
        #print(data['date'])


        y = data['nextweek_'+ current_currency]
        
        
        X = data.loc[:, e['needed_features'] ]

       




        X_train = X[1:]
        y_train = y[1:]
        X_test = X[:1]
        y_test = y[:1]




        clf=RandomForestRegressor(n_estimators=n_estimators, n_jobs =-1, min_samples_leaf=min_samples_leaf,
                                     max_features=max_features, max_samples = max_samples)

        clf.fit(X_train,y_train)
        y_pred=clf.predict(X_test)


        y_test = y_test.values




        if y_pred[0] > cut_off  or y_pred[0] < (-1*cut_off) :
            number_of_trades = number_of_trades + 1
            #print(y_pred[0], y_test[0])


            if (y_pred[0] > 0 and y_test[0] == 1) or (y_pred[0] < 0 and y_test[0]) == -1:
                number_of_accurate_prediction = number_of_accurate_prediction +1
                


    e['accuracy_score'] =  100*number_of_accurate_prediction/number_of_trades
    print(e['accuracy_score'])
    
    

GBPUSD
60.0
GBPUSD
60.0
GBPUSD
60.0
GBPUSD
60.0
GBPUSD
60.0
USDCAD
60.0
USDCAD
40.0
USDCAD
20.0
USDCAD
60.0
AUDUSD
60.0
NZDUSD
40.0
NZDUSD
40.0
NZDUSD
60.0
GBPCHF
40.0
GBPCHF
60.0
EURCHF
60.0
CHFJPY
60.0
CADCHF
40.0
AUDCHF
80.0
AUDCHF
40.0
AUDCHF
40.0
NZDCHF
60.0
NZDCHF
80.0
EURGBP
60.0
GBPJPY
60.0
GBPJPY
40.0
GBPJPY
40.0
GBPJPY
40.0
GBPCAD
40.0
GBPAUD
60.0
GBPAUD
60.0
GBPAUD
80.0
GBPAUD
60.0
GBPAUD
80.0
EURCAD
80.0
EURCAD
60.0
EURAUD
80.0
EURAUD
80.0
EURAUD
80.0
EURAUD
40.0
EURNZD
80.0
EURNZD
100.0
EURNZD
80.0
EURNZD
60.0
CADJPY
60.0
CADJPY
60.0
CADJPY
60.0
CADJPY
60.0
CADJPY
60.0
AUDJPY
40.0
AUDJPY
80.0
NZDJPY
60.0
NZDJPY
100.0
NZDJPY
40.0
AUDCAD
20.0
NZDCAD
60.0
NZDCAD
40.0
NZDCAD
80.0
NZDCAD
60.0
AUDNZD
80.0
AUDNZD
60.0
AUDNZD
60.0
AUDNZD
80.0


In [6]:
next_week_trades =[]

for f in currency_pairs:
    
    current_currency = f.replace('10080','')
    current_currency_list =[]
    current_currency_best_trade = None
    
    for g in currency_pair_detail:
        
        
        
        if g['currency_name'] == current_currency:
            
            current_currency_list.append(g)
            
    
    current_currency_list.sort(key=lambda x: x['week'], reverse=False)
    
    for h in current_currency_list:
        
        if h['accuracy_score'] > 99.9:
            
            current_currency_best_trade = h
            
            break
            

    if current_currency_best_trade == None:
       
        for i in current_currency_list:

            if i['accuracy_score'] > 79.9:
                
                current_currency_best_trade = i
                
                break
    
    
    if current_currency_best_trade != None:
        
        next_week_trades.append(current_currency_best_trade)
    
    
        
            
print(next_week_trades)            
            


[{'currency_name': 'AUDCHF', 'week': 10, 'needed_features': ['USDJPY'], 'accuracy_score': 80.0}, {'currency_name': 'NZDCHF', 'week': 50, 'needed_features': ['AUDCAD', 'EURCHF', 'AUDUSD', 'AUDJPY', 'GBPCAD'], 'accuracy_score': 80.0}, {'currency_name': 'GBPAUD', 'week': 20, 'needed_features': ['USDJPY', 'CHFJPY', 'AUDJPY', 'EURUSD', 'AUDUSD', 'EURCHF'], 'accuracy_score': 80.0}, {'currency_name': 'EURCAD', 'week': 20, 'needed_features': ['USDCHF'], 'accuracy_score': 80.0}, {'currency_name': 'EURAUD', 'week': 10, 'needed_features': ['AUDCAD'], 'accuracy_score': 80.0}, {'currency_name': 'EURNZD', 'week': 20, 'needed_features': ['AUDCAD', 'EURCHF'], 'accuracy_score': 100.0}, {'currency_name': 'AUDJPY', 'week': 50, 'needed_features': ['CADCHF', 'NZDCAD'], 'accuracy_score': 80.0}, {'currency_name': 'NZDJPY', 'week': 30, 'needed_features': ['EURCHF', 'AUDCHF', 'USDCAD'], 'accuracy_score': 100.0}, {'currency_name': 'NZDCAD', 'week': 20, 'needed_features': ['EURCHF', 'AUDCHF', 'GBPAUD', 'EURAUD']

In [7]:
# creating a dataframe for nextweek trades and saving it in csv

name =[]
week = []
needed_features = []
accuracy =[]

for j in next_week_trades:
    
    name.append(j['currency_name'])
    week.append(j['week'])
    needed_features.append(j['needed_features'])
    accuracy.append(j['accuracy_score'])
    
    
next_week_trades_dataframe = pd.DataFrame()
next_week_trades_dataframe['names']=name
next_week_trades_dataframe['week'] =week
next_week_trades_dataframe['needed_features'] = needed_features
next_week_trades_dataframe['accuracy']=accuracy


print(next_week_trades_dataframe)

next_week_trades_dataframe.to_csv("files/next_week_trades.csv")

    names  week                                   needed_features  accuracy
0  AUDCHF    10                                          [USDJPY]      80.0
1  NZDCHF    50          [AUDCAD, EURCHF, AUDUSD, AUDJPY, GBPCAD]      80.0
2  GBPAUD    20  [USDJPY, CHFJPY, AUDJPY, EURUSD, AUDUSD, EURCHF]      80.0
3  EURCAD    20                                          [USDCHF]      80.0
4  EURAUD    10                                          [AUDCAD]      80.0
5  EURNZD    20                                  [AUDCAD, EURCHF]     100.0
6  AUDJPY    50                                  [CADCHF, NZDCAD]      80.0
7  NZDJPY    30                          [EURCHF, AUDCHF, USDCAD]     100.0
8  NZDCAD    20                  [EURCHF, AUDCHF, GBPAUD, EURAUD]      80.0
9  AUDNZD    15                                          [USDCHF]      80.0


In [8]:
control_dataframe = pd.read_csv('control.csv' ) 
control_dataframe['feature'].iloc[:1] = 1


print(control_dataframe)
control_dataframe.to_csv('control.csv', index=False)


   classify  corelation  feature  predict  drawdownup
0         1           1        1        0           0
